## Thermal Face Detection

In [1]:
!pip install PyYAML==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 34.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [2]:
!git clone https://github.com/WongKinYiu/yolov7

Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (467/467), done.


In [3]:
%cd yolov7

/content/yolov7


In [4]:
# Download dataset from roboflow (https://universe.roboflow.com/eceproject/face-recognition-oev8w)

In [20]:
%%writefile data.yaml

train: /content/yolov7/train/images/
val: /content/yolov7/valid/images/

nc: 1
names: ['face']

Overwriting data.yaml


In [18]:
%%writefile  data/hyp.scratch.p5.yaml

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.2  # image translation (+/- fraction)
scale: 0.4  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.2  # image mixup (probability)
copy_paste: 0.0  # image copy paste (probability)
paste_in: 0.2  # image copy paste (probability), use 0 for faster training
loss_ota: 1 # use ComputeLossOTA, use 0 for faster training

Overwriting data/hyp.scratch.p5.yaml


In [8]:
# Download weights from https://github.com/WongKinYiu/yolov7

In [9]:
!curl -L "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x.pt" > yolov7x.pt;

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  136M  100  136M    0     0  9208k      0  0:00:15  0:00:15 --:--:-- 13.6M


In [10]:
!nvidia-smi

Sat Nov 19 20:41:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python train.py --workers 8 --adam --device 0 --batch-size 42 --single-cls --image-weights --cache-images --data data.yaml --img 320 --cfg cfg/training/yolov7x.yaml --weights 'yolov7x.pt' --name yolov7 --hyp data/hyp.scratch.p5.yaml --epochs=100

In [ ]:
%%writefile data.yaml

val: /content/yolov7/test/images/

nc: 1
names: ['face']

In [22]:
!python test.py --device 0 --batch-size 16 --single-cls --data test.yaml --img 320 --weights 'runs/train/yolov7/weights/best.pt' 

Namespace(augment=False, batch_size=16, conf_thres=0.001, data='data.yaml', device='0', exist_ok=False, img_size=320, iou_thres=0.65, name='exp', no_trace=False, project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=True, task='val', v5_metric=False, verbose=False, weights=['runs/train/yolov7/weights/best.pt'])
YOLOR 🚀 v0.1-115-g072f76c torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
IDetect.fuse
Model Summary: 362 layers, 70782444 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
val: Scanning '/content/yolov7/test/labels' images and labels... 104 found, 



---

